In [1]:
import torch
import torch.nn as nn
import numpy as np

import yfinance as yf
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from modules.scalers import GroupByScaler
from sklearn.preprocessing import MaxAbsScaler
import json
import torch

from modules.environment import MultiPeriodPortfolioOptimizationEnv
from modules.models import DRLAgent

sns.set()

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [2]:
config_file_name = "config.json"
with open(config_file_name, "r") as jsonfile:
    config_data = json.load(jsonfile)

In [3]:
sp_100 = config_data["tickers"]["America"]["SP100"]
NUM_ASSETS = len(sp_100)
len(sp_100)

101

In [4]:
start_date = config_data["timeframe"]["bull_market"]["start_date"]
end_date = config_data["timeframe"]["bull_market"]["end_date"]
data_interval = config_data["data_interval"]
test_ratio = config_data["train_test_ratio"]
random_seed = config_data["random_state_seed"]

In [5]:
portfolio_raw_df = yf.download(tickers=sp_100, start=start_date, end=end_date, interval=data_interval)

portfolio_raw_df.fillna(method="bfill", inplace=True)
portfolio_raw_df = portfolio_raw_df.stack(level=1).rename_axis(["Date", "Ticker"]).reset_index(level=1)
portfolio_raw_df = portfolio_raw_df.drop("Adj Close", axis=1)
portfolio_raw_df.columns.name = None
portfolio_raw_df = portfolio_raw_df.reset_index()
portfolio_raw_df.Date = portfolio_raw_df.Date.astype(str)
portfolio_raw_df.columns = ["date", "tic", "close", "high", "low", "open", "volume"]
portfolio_raw_df = portfolio_raw_df[["date", "tic", "close", "high", "low", "volume"]]

df_portfolio_raw_train, df_portfolio_raw_test = train_test_split(portfolio_raw_df, test_size=test_ratio, shuffle=False, random_state=random_seed)
df_portfolio_train = GroupByScaler(by="tic", scaler=MaxAbsScaler).fit_transform(df_portfolio_raw_train)
df_portfolio_test = GroupByScaler(by="tic", scaler=MaxAbsScaler).fit_transform(df_portfolio_raw_test)


[*********************100%%**********************]  101 of 101 completed
/var/folders/02/rmgfx9wj0s99nn4tzfthtrnc0000gn/T/ipykernel_16627/541920902.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  portfolio_raw_df.fillna(method="bfill", inplace=True)
/var/folders/02/rmgfx9wj0s99nn4tzfthtrnc0000gn/T/ipykernel_16627/541920902.py:4: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  portfolio_raw_df = portfolio_raw_df.stack(level=1).rename_axis(["Date", "Ticker"]).reset_index(level=1)


In [6]:
# SETTING HYPERPARAMETERS
FEATURES = ["close", "high", "low", "volume"]
N = config_data["lookback_window"]
T = config_data["multi_step_horizon"]
NUM_FEATURES = len(FEATURES)
experiment_type = "HYBRID_TRANSFORMER" 
N, T, NUM_ASSETS, NUM_FEATURES

(24, 3, 101, 4)

In [7]:
train_environment = MultiPeriodPortfolioOptimizationEnv(
    df_portfolio_train,
    initial_amount=100000,
    comission_fee_pct=0.0025,
    time_window=N,
    multi_period_horizon=T,
    features=FEATURES,
    normalize_df=None,
    is_train_mode=True,
    experiment_type=experiment_type
)

In [8]:
class ConvolutionalEmbedding(nn.Module):
    def __init__(self, in_channels, out_channels, window_length=24):
        super(ConvolutionalEmbedding, self).__init__()
        self.convolution = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=(1, window_length))

    def forward(self, X):
        conv_tokens = self.convolution(X)
        return conv_tokens

class MultiPeriodConvAttentionNetwork(nn.Module):
    def __init__(self, num_stocks, num_features, W, T, num_layers=10, device="cpu"):
        super(MultiPeriodConvAttentionNetwork, self).__init__()
        self.device = device
        num_assets = num_stocks + 1
        self.W = W
        self.T = T
        self.state_conv_tokenization = ConvolutionalEmbedding(in_channels=num_features,
                                                         out_channels=T,
                                                         window_length=W)
        self.last_action_tokenization = ConvolutionalEmbedding(in_channels=1,
                                                               out_channels=1,
                                                               window_length=1)
        self.encoder_state = nn.TransformerEncoder(encoder_layer=nn.TransformerEncoderLayer(d_model=num_assets, nhead=1, batch_first=True), num_layers=num_layers)
        self.encoder_last_action = nn.TransformerEncoder(encoder_layer=nn.TransformerEncoderLayer(d_model=num_assets, nhead=1, batch_first=True), num_layers=num_layers)
        self.decoder = nn.TransformerDecoder(decoder_layer=nn.TransformerDecoderLayer(d_model=num_assets, nhead=1, batch_first=True), num_layers=num_layers)
        
        self.norm = nn.Softmax(dim=2)

    def forward(self, observation, last_action):
        mu = self.mu(observation, last_action)
        action = mu.cpu().detach().numpy().squeeze(0)
        return action

    def mu(self, state, last_action):
        if isinstance(state, np.ndarray):
            state = torch.from_numpy(state)
        state = state.to(self.device).float()

        if isinstance(last_action, np.ndarray):
            last_action = torch.from_numpy(last_action)
        last_action = last_action.to(self.device).float()
        
        state_tokens = self.state_conv_tokenization(state)
        last_action = last_action.unsqueeze(1)
        last_action_tokens = self.last_action_tokenization(last_action)
        state_tokens = state_tokens.squeeze(3)
        last_action_tokens = last_action_tokens.squeeze(1)

        
        cash_bias = self.process_last_action(last_action)
        state_tokens = torch.cat([state_tokens, cash_bias], dim=2)
        state_embeddings = self.encoder_state(state_tokens)
        last_action_embeddings = self.encoder_last_action(last_action_tokens)
        embeddings = torch.cat([state_embeddings, last_action_embeddings], dim=1)
        logits = self.decoder(tgt=last_action_tokens, memory=embeddings)
        portfolio_weights = self.norm(logits)
        return portfolio_weights
    
    def process_last_action(self, last_action):
        batch_size = last_action.shape[0]
        cash_allocation = last_action[:, :, :, 0]
        cash_allocation = cash_allocation.reshape((batch_size, self.T, 1))
        return cash_allocation

In [18]:
# policy = MultiPeriodConvAttentionNetwork(num_stocks=NUM_ASSETS,
#                                          num_features=NUM_FEATURES,
#                                          W=N,
#                                          T=T)

/opt/homebrew/lib/python3.12/site-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.num_heads is odd
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [19]:
# state = torch.rand(1, NUM_FEATURES, NUM_ASSETS, N)
# last_action = torch.rand(1, T, NUM_ASSETS+1)
# state.shape, last_action.shape

(torch.Size([1, 4, 101, 24]), torch.Size([1, 3, 102]))

In [23]:
# policy(state, last_action).min(1)

array([0.00050308, 0.00039293, 0.00058882], dtype=float32)

In [9]:
# set PolicyGradient parameters
model_kwargs = {
    "lr": 0.001,
    "policy": MultiPeriodConvAttentionNetwork,
    "multi_period_horizon": T
}

# here, we can set EIIE's parameters
policy_kwargs = {
    "num_features": NUM_FEATURES,
    "num_stocks": NUM_ASSETS,
    "W": N,
    "T": T
}

In [10]:
model = DRLAgent(train_environment).get_model("pg", device, model_kwargs, policy_kwargs)

/opt/homebrew/lib/python3.12/site-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.num_heads is odd
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [11]:
print("TRAINING AGENT.....")
DRLAgent.train_model(model, episodes=50)

TRAINING AGENT.....


  0%|          | 0/50 [00:00<?, ?it/s]

Initial portfolio value:100000
Final portfolio value: 2004674.2109971049
Final accumulative portfolio value: 20.04674210997105
Maximum DrawDown: -0.7717208128857289
Sharpe ratio: 5.839784640611192


  2%|▏         | 1/50 [00:05<04:39,  5.71s/it]

Initial portfolio value:100000
Final portfolio value: 2072447.1752922295
Final accumulative portfolio value: 20.724471752922295
Maximum DrawDown: -0.7697776594579664
Sharpe ratio: 5.904474435103387


...

 96%|█████████▌| 48/50 [04:22<00:10,  5.47s/it]

Initial portfolio value:100000
Final portfolio value: 2440721.102322946
Final accumulative portfolio value: 24.407211023229458
Maximum DrawDown: -0.7576869208608812
Sharpe ratio: 6.1586368430498295


 98%|█████████▊| 49/50 [04:28<00:05,  5.53s/it]

Initial portfolio value:100000
Final portfolio value: 2440453.999837825
Final accumulative portfolio value: 24.40453999837825
Maximum DrawDown: -0.7576793011065849
Sharpe ratio: 6.1584896925508446


100%|██████████| 50/50 [04:33<00:00,  5.47s/it]


In [12]:
print("PERSISTING MODEL.....")
torch.save(model.train_policy.state_dict(), f"models/policy_{experiment_type}.pt")

PERSISTING MODEL.....


In [13]:
print("TESTING.....")
MEIIE_results = {
    "training": train_environment._asset_memory["final"],
    "test": {}
}

TESTING.....


In [14]:
test_environment = MultiPeriodPortfolioOptimizationEnv(
    df_portfolio_test,
    initial_amount=100000,
    comission_fee_pct=0.0025,
    time_window=N,
    multi_period_horizon=T,
    features=FEATURES,
    normalize_df=None,
    is_train_mode=False,
    experiment_type=experiment_type
)

In [15]:
policy = MultiPeriodConvAttentionNetwork(num_stocks=NUM_ASSETS,
                                         num_features=NUM_FEATURES,
                                         W=N,
                                         T=T)

/opt/homebrew/lib/python3.12/site-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.num_heads is odd
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [16]:
policy.load_state_dict(torch.load(f"models/policy_{experiment_type}.pt"))

<All keys matched successfully>

In [17]:
DRLAgent.DRL_validation(model=model, test_env=test_environment, policy=policy)

Initial portfolio value:100000
Final portfolio value: 533552.0558170116
Final accumulative portfolio value: 5.335520558170116
Maximum DrawDown: -0.39917030214508575
Sharpe ratio: 5.6634839201682325


In [18]:
MEIIE_results["test"]["value"] = test_environment._asset_memory["final"]

In [19]:
drl_portfolio_performance = test_environment._asset_memory["final"][1:]
len(drl_portfolio_performance)

70

In [20]:
drl_portfolio_performance = [(x/100000)-1 for x in drl_portfolio_performance]

In [21]:
performance_dataset = pd.read_csv("data/processed/performances_sp100.csv", index_col=0)

In [22]:
performance_dataset["DRL_HYBRID_TRANSFORMER"] = drl_portfolio_performance

In [23]:
performance_dataset.to_csv("data/processed/performances_sp100.csv")